In [3]:
import ollama
from tqdm.auto import tqdm
from human_eval.data import write_jsonl, read_problems

In [14]:
LLM_MODEL_NAME = "deepseek-coder"

In [32]:
problems = read_problems()
print(f"Loaded {len(problems)} problems")

Loaded 164 problems


In [5]:
# we can generate more than one candidate per task
# later one pass@1, pass@10... will be used to evaluate the model
num_samples_per_task = 1

In [51]:
foo_task = '\ndef generate_foo():\n    """\n    Return the string foo.\n\n    For example:\n    foo() => foo\n    """\n'
print(foo_task)


def generate_foo():
    """
    Return the string foo.

    For example:
    foo() => foo
    """



In [52]:
def generate_one_completion(prompt):
    response = ollama.chat(
        model=LLM_MODEL_NAME,
        messages=[
            {
                'role': 'system',
                'content': 'You are a software developer. You need to just write how the code continues.'
            },
            {
                'role': 'user',
                'content': f"Continue the code generation for the followind function:\n{foo_task}"
            },
            {
                'role': 'assistant',
                'content': "return 'foo'"
            },
            {
                'role': 'user',
                'content': f"Continue the code generation for the followind function:\n{prompt}"
            },
        ],
        options={'temperature': 0.25}
    )
    return response['message']['content']

In [46]:
# take a subset of 5 problems to test the model, problems is a dict
#problems = {k: problems[k] for k in list(problems.keys())[:1]}

In [33]:
results= []
for task_id in tqdm(problems):
    for _ in range(num_samples_per_task):
        results.append({
            'task_id': task_id,
            'completion': generate_one_completion(problems[task_id]['prompt']),
        })
    break

  0%|          | 0/164 [00:00<?, ?it/s]

In [35]:
print(results[0]["completion"])

from typing import List
import math
def has_close_elements(numbers:List[float], threshold: float)->bool :   # Define the function with correct type hinting and docstring 
    """ Check if in given list of numbers, are any two numbers closer to each other than
        Given Threshold. Return True when there is a pair that's within 'threshold', else False."""    
        
    for ix1 , num_a in enumerate(numbers):  # Iterate over the elements and their indices  
       for ix2, num_b in enumerate(numbers[ix1+1:]):      # Start from next index after current element to avoid duplicate comparisons.   
           if math.isclose (abs(num_a - num_b),threshold ,rel_tol=0.5  ) :   # Use is close function for floating point comparison, with tolerance set at half of the threshold value    
               return True      # Return true as soon found a pair that's within 'Threshold'.   
       if ix1 == len(numbers)-2:        # If it reached last element in list then break out from loop.  
        

In [36]:
# fill the rest of the problems with 'return None'
for task_id in problems:
    if task_id in results:
        continue
    for _ in range(num_samples_per_task):
        results.append({
            'task_id': task_id,
            'completion': 'return None',
        })

In [37]:
write_jsonl(f"submission_ollama_{LLM_MODEL_NAME}.jsonl", results)
# Now run: $ evaluate_functional_correctness samples.jsonl